In [1]:
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# metrics = {'mse':0, 'auc':1, 'll':2, 'acc':3, 'perp':4, 'npmi':5, 'shuffle':6, 'shuffle_std':7}
metrics = {'mse':0, 'auc':1, 'll':2, 'acc':3, 'perp':4, 'npmi':5, 'shuffle':6}
classification_settings = {'peerread', 'yelp', 'amazon_binary', 'framing_corpus'}

In [3]:
def get_best_results_regression(exp_results, data, model):
    best_npmi=-1e7
    best_npmi_config=None
    best_mse=1e7
    best_mse_config=None
    best_perp = 1e8
    best_perp_config=None
    

    for config, res in exp_results[data][model].items():
        mean_results = res.mean(axis=0)
        mse=mean_results[metrics['mse']]
        perp=mean_results[metrics['perp']]
        npmi=mean_results[metrics['npmi']]
        if mse < best_mse:
            best_mse=mse
            best_mse_config=config
        if perp < best_perp:
            best_perp=perp
            best_perp_config=config
        if npmi > best_npmi:
            best_npmi=npmi
            best_npmi_config=config
    
    return best_npmi_config, best_mse_config, best_perp_config

def get_best_results_classification(exp_results, data, model):
    best_npmi=-1e7
    best_npmi_config=None
    best_acc=0.
    best_acc_config=None
    best_perp = 1e8
    best_perp_config=None

    for config, res in exp_results[data][model].items():
        mean_results = res.mean(axis=0)
        acc=mean_results[metrics['acc']]
        perp=mean_results[metrics['perp']]
        npmi=mean_results[metrics['npmi']]
        if acc > best_acc:
            best_acc=acc
            best_acc_config=config
        if perp < best_perp:
            best_perp=perp
            best_perp_config=config
        if npmi > best_npmi:
            best_npmi=npmi
            best_npmi_config=config
    
    return best_npmi_config, best_acc_config, best_perp_config

In [9]:
models = ['hstm', 'hstm-nobeta', 'hstm-all']
stm_models = ['stm', 'stm+bow']

datasets = ['amazon', 'amazon_binary', 'peerread', 'yelp']

out = '../out/'

Cs = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
Ctopics = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
num_folds = 5
n_metrics = len(metrics.keys())
exp_results = {data:{model:{(C,Ct):np.zeros((num_folds,n_metrics)) for C in Cs for Ct in Ctopics}
                     for model in models}for data in datasets}
for data in datasets:
    for sm in stm_models:
        exp_results[data].update({sm:{(C,1e-6):np.zeros((num_folds,n_metrics)) for C in Cs}})

for data in datasets:
    datadir = data + '.lda_pretrained'
    for model in stm_models + models:
        for C in Cs:
            if model in stm_models:
                for e_idx in range(num_folds):
                    resfile= out + datadir + '/' + model + '.result.split'+str(e_idx) + '.setting=' + str((C,1e-6))
                    results = np.load(resfile + '.npy')
                    exp_results[data][model][(C,1e-6)][e_idx]=results
            else:
                for Ct in Ctopics:
                    for e_idx in range(num_folds):
                        resfile= out + datadir + '/' + model + '.result.split'+str(e_idx) + '.setting=' + str((C,Ct))
                        results = np.load(resfile + '.npy')
                        exp_results[data][model][(C,Ct)][e_idx]=results


In [10]:
for data in datasets:
    print("*"*20)
    print("Dataset:", data)
    for model in models + stm_models:
        if data not in classification_settings:
            c_nmpi, c_mse, c_perp = get_best_results_regression(exp_results, data, model)
            results = exp_results[data][model][c_mse].mean(axis=0)
            stds = exp_results[data][model][c_mse].std(axis=0)
            results = np.round(results, 3)
            stds = np.round(stds, 3)
            
            print("*"*10)
            print("Model:", model, c_mse)
            print("MSE:", (results[metrics['mse']],
                  stds[metrics['mse']]), 
                  "NPMI:", results[metrics['npmi']], 
                  'Perplexity:', results[metrics['perp']],
                    'Shuffle loss:', results[metrics['shuffle']])
        else:
            c_nmpi, c_mse, c_perp = get_best_results_classification(exp_results, data, model)
            results = exp_results[data][model][c_mse].mean(axis=0)
            stds = exp_results[data][model][c_mse].std(axis=0)
            results = np.round(results, 3)
            stds = np.round(stds, 3)
            print("*"*10)
            print("Model:", model, c_mse)
            print("Accuracy:", (results[metrics['acc']], stds[metrics['acc']]),
                  "AUC:", (results[metrics['auc']], stds[metrics['auc']]),
                  "Log loss:", (results[metrics['ll']], stds[metrics['ll']]),
                  "NPMI:", results[metrics['npmi']], 
                  'Perplexity:', results[metrics['perp']],
                 'Shuffle loss:', results[metrics['shuffle']])

********************
Dataset: amazon
**********
Model: hstm (5e-06, 1e-05)
MSE: (0.698, 0.035) NPMI: 15.939 Perplexity: 297.62 Shuffle loss: 0.0
**********
Model: hstm-nobeta (0.0001, 1e-06)
MSE: (0.704, 0.035) NPMI: 15.965 Perplexity: 312.22 Shuffle loss: 0.0
**********
Model: hstm-all (5e-06, 1e-05)
MSE: (0.702, 0.034) NPMI: 15.843 Perplexity: 282.926 Shuffle loss: 0.0
**********
Model: stm (5e-05, 1e-06)
MSE: (0.931, 0.046) NPMI: 15.847 Perplexity: 313.459 Shuffle loss: 0.0
**********
Model: stm+bow (5e-06, 1e-06)
MSE: (0.716, 0.035) NPMI: 15.935 Perplexity: 357.34 Shuffle loss: 0.0
********************
Dataset: amazon_binary
**********
Model: hstm (5e-06, 1e-06)
Accuracy: (0.927, 0.004) AUC: (0.926, 0.005) Log loss: (0.187, 0.003) NPMI: 9.886 Perplexity: 30.464 Shuffle loss: 0.0
**********
Model: hstm-nobeta (1e-06, 1e-06)
Accuracy: (0.91, 0.003) AUC: (0.919, 0.004) Log loss: (0.215, 0.004) NPMI: 9.65 Perplexity: 31.372 Shuffle loss: 0.0
**********
Model: hstm-all (5e-06, 1e-06)
Ac

In [7]:
models = ['hstm-all']
stm_models = ['stm', 'stm+bow']
data = 'framing_corpus'
topics = ['deathpenalty', 'immigration', 'guncontrol', 'samesex']

out = '../out/'

Cs = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
Ctopics = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
num_folds = 5
n_metrics = len(metrics.keys())
exp_results = {topic:{model:{(C,Ct):np.zeros((num_folds,n_metrics)) for C in Cs for Ct in Ctopics}
                     for model in models} for topic in topics}
for topic in topics:
    for sm in stm_models:
        exp_results[topic].update({sm:{(C,1e-6):np.zeros((num_folds,n_metrics)) for C in Cs}})

for topic in topics:
    datadir = data + '_' + topic + '.lda_pretrained'
    for model in stm_models + models:
        for C in Cs:
            if model in stm_models:
                for e_idx in range(num_folds):
                    resfile= out + datadir + '/' + model + '.result.split'+str(e_idx) + '.setting=' + str((C,1e-6))
                    results = np.load(resfile + '.npy')
                    exp_results[topic][model][(C,1e-6)][e_idx]=results
            else:
                for Ct in Ctopics:
                    for e_idx in range(num_folds):
                        resfile= out + datadir + '/' + model + '.result.split'+str(e_idx) + '.setting=' + str((C,Ct))
                        results = np.load(resfile + '.npy')
                        exp_results[topic][model][(C,Ct)][e_idx]=results


In [8]:
for topic in topics:
    print("*"*20)
    print("Topic:", topic)
    for model in models + stm_models:
        if data not in classification_settings:
            c_nmpi, c_mse, c_perp = get_best_results_regression(exp_results, topic, model)
            results = exp_results[topic][model][c_mse].mean(axis=0)
            stds = exp_results[topic][model][c_mse].std(axis=0)
            results = np.round(results, 3)
            stds = np.round(stds, 3)
            
            print("*"*10)
            print("Model:", model, c_mse)
            print("MSE:", (results[metrics['mse']],
                  stds[metrics['mse']]), 
                  "NPMI:", results[metrics['npmi']], 
                  'Perplexity:', results[metrics['perp']],
                    'Shuffle loss:', results[metrics['shuffle']])
        else:
            c_nmpi, c_mse, c_perp = get_best_results_classification(exp_results, topic, model)
            results = exp_results[topic][model][c_mse].mean(axis=0)
            stds = exp_results[topic][model][c_mse].std(axis=0)
            results = np.round(results, 3)
            stds = np.round(stds, 3)
            print("*"*10)
            print("Model:", model, c_mse)
            print("Accuracy:", (results[metrics['acc']], stds[metrics['acc']]),
                  "AUC:", (results[metrics['auc']], stds[metrics['auc']]),
                  "Log loss:", (results[metrics['ll']], stds[metrics['ll']]),
                  "NPMI:", results[metrics['npmi']], 
                  'Perplexity:', results[metrics['perp']],
                 'Shuffle loss:', results[metrics['shuffle']])

********************
Topic: deathpenalty
**********
Model: hstm-all (5e-06, 5e-06)
Accuracy: (0.785, 0.023) AUC: (0.859, 0.01) Log loss: (0.469, 0.013) NPMI: 4.041 Perplexity: 322.145 Shuffle loss: 0.0
**********
Model: stm (5e-05, 1e-06)
Accuracy: (0.689, 0.014) AUC: (0.747, 0.011) Log loss: (0.596, 0.009) NPMI: 4.243 Perplexity: 358.056 Shuffle loss: 0.0
**********
Model: stm+bow (5e-05, 1e-06)
Accuracy: (0.759, 0.014) AUC: (0.824, 0.009) Log loss: (0.527, 0.008) NPMI: 4.262 Perplexity: 314.19 Shuffle loss: 0.0
********************
Topic: immigration
**********
Model: hstm-all (1e-05, 1e-06)
Accuracy: (0.804, 0.014) AUC: (0.866, 0.009) Log loss: (0.439, 0.014) NPMI: 4.105 Perplexity: 472.945 Shuffle loss: 0.0
**********
Model: stm (1e-05, 1e-06)
Accuracy: (0.721, 0.016) AUC: (0.775, 0.011) Log loss: (0.547, 0.01) NPMI: 4.858 Perplexity: 636.971 Shuffle loss: 0.0
**********
Model: stm+bow (1e-06, 1e-06)
Accuracy: (0.775, 0.023) AUC: (0.839, 0.008) Log loss: (0.488, 0.011) NPMI: 4.047 

/Users/dhanyasridhar/opt/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:193: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
